In [3]:
import gensim
import pandas as pd
from nltk.corpus import wordnet
import nltk
from nltk.corpus import stopwords
import numpy as np

In [10]:
col_names = ["user", "date", "rating", "title", "content"]
df = pd.read_csv('C:\\Users\ACER\\Documents\\ta\\Review Per Hotel\\Bintang 4\\riuny.csv', sep=';')

In [11]:
df = df.rename(columns={'user ': 'user'})
df = df.rename(columns={'date ': 'date'})
df = df.rename(columns={'rating ': 'rating'})
df = df.rename(columns={'title ': 'title'})
df = df.rename(columns={'content ': 'content'})

In [12]:
df.head()

,user,date,rating,title,content
0,ander869,Jul-22,40,Very nice hotel!,"Very very good position, 5 mins to Times Squar..."
1,AnnArborMichigan,Aug-21,50,My new go-to hotel in NYC,I picked this hotel because it was comparative...
2,D&R Travels,Jul-22,50,Riu Location Location Location..!!!,We recently spent our 4th July weekend trip in...
3,Traveler64701,Jul-22,40,Beautiful hotel,Only issue is there are 2 Riu with similar nam...
4,northeastfoodgal,Jul-22,30,Comfortable but fine tuning needed.,We booked our trip through British Airways and...


In [13]:
df = df.rename(columns={' content ': 'content'})
df = df.rename(columns={' user ': 'user'})
df = df.rename(columns={' date ': 'date'})
df = df.rename(columns={' rating ': 'rating'})
df = df.rename(columns={' title ': 'title'})

In [14]:
df['title']=df['title'].str.lower()
df['content']=df['content'].str.lower()

In [15]:
df.head()

,user,date,rating,title,content
0,ander869,Jul-22,40,very nice hotel!,"very very good position, 5 mins to times squar..."
1,AnnArborMichigan,Aug-21,50,my new go-to hotel in nyc,i picked this hotel because it was comparative...
2,D&R Travels,Jul-22,50,riu location location location..!!!,we recently spent our 4th july weekend trip in...
3,Traveler64701,Jul-22,40,beautiful hotel,only issue is there are 2 riu with similar nam...
4,northeastfoodgal,Jul-22,30,comfortable but fine tuning needed.,we booked our trip through british airways and...


In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
" ".join(stopwords.words('english'))
stop_words = set(stopwords.words('english'))
def remove_stop(x):
    return " ".join([word for word in str(x).split() if word not in stop_words])
df['title']=df['title'].apply(lambda x : remove_stop(x))
def remove_stop(x):
    return " ".join([word for word in str(x).split() if word not in stop_words])
df['content']=df['content'].apply(lambda x : remove_stop(x))

In [18]:
df.describe()

,rating
count,2410.000000
mean,42.099585
std,10.990318
min,10.000000
25%,40.000000
50%,50.000000
75%,50.000000
max,50.000000


In [19]:
review_text = df.content.apply(gensim.utils.simple_preprocess)
review_text

0       [good, position, mins, times, square, walk, pe...
1       [picked, hotel, comparatively, inexpensive, lo...
2       [recently, spent, th, july, weekend, trip, nyc...
3       [issue, riu, similar, names, block, apart, car...
4       [booked, trip, british, airways, whilst, infor...
                              ...                        
2405    [stayed, riu, advertised, soft, open, food, ba...
2406    [stayed, riu, last, night, the, receptionist, ...
2407    [stayed, spring, break, week, year, made, rese...
2408    [hotel, brand, new, basically, appointed, obvi...
2409    [stay, nights, last, minute, booking, front, d...
Name: content, Length: 2410, dtype: object

In [20]:
model = gensim.models.Word2Vec()

In [21]:
model.build_vocab(review_text,progress_per=1000)

In [22]:
model.epochs

5

In [23]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(698649, 859565)

In [24]:
model.save("./Word2Vec-tripadvisor-riu-reviews-short.model")

# Dimensi Sincerity

In [44]:
trait1=model.wv.most_similar("honest", topn=35)
simtrait1=[]
for i in range(0,len(trait1)):
    if wordnet.synsets(trait1[i][0], pos = wordnet.ADJ) != []:
        simtrait1.append(trait1[i][0])
    else:
        continue
sim1=list(simtrait1)
print(sim1)        

['running', 'giving', 'true', 'known', 'similar', 'important', 'glad', 'piano', 'certain', 'meet']


In [48]:
synonyms1 = []
for syn in wordnet.synsets("honest", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms1.append(l.name()) 
syn1 = list(synonyms1)        
print(syn1)      

['honest', 'honorable', 'honest', 'dependable', 'honest', 'reliable', 'true', 'honest', 'honest', 'good', 'honest', 'honest', 'fair']


In [49]:
for a in range(0,len(sim1)):
    for b in range(0,len(syn1)):
        try:
            tes=model.wv.similarity(w1=sim1[a],w2=syn1[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim1[a], syn1[b], tes)
        else: 
            pass
        

running honest 0.9954696
running honorable 0.9954696
running honest 0.9954696
running dependable 0.9954696
running honest 0.9954696
running reliable 0.9954696
running true 0.99301016
running honest 0.9954696
running honest 0.9954696
running good 0.5541998
running honest 0.9954696
running honest 0.9954696
running fair 0.9860444
giving honest 0.99520063
giving honorable 0.99520063
giving honest 0.99520063
giving dependable 0.99520063
giving honest 0.99520063
giving reliable 0.99520063
giving true 0.98960567
giving honest 0.99520063
giving honest 0.99520063
giving good 0.5790652
giving honest 0.99520063
giving honest 0.99520063
giving fair 0.98492646
true honest 0.993679
true honorable 0.993679
true honest 0.993679
true dependable 0.993679
true honest 0.993679
true reliable 0.993679
true true 1.0
true honest 0.993679
true honest 0.993679
true good 0.5652817
true honest 0.993679
true honest 0.993679
true fair 0.99186456
known honest 0.9935275
known honorable 0.9935275
known honest 0.993527

In [51]:
trait6=model.wv.most_similar("real", topn=40)
simtrait6=[]
for i in range(0,len(trait6)):   
    if wordnet.synsets(trait6[i][0], pos = wordnet.ADJ) != []:
        simtrait6.append(trait6[i][0])
    else:
        continue
sim6=list(simtrait6)
print(sim6) 

['difficult', 'chaotic', 'personal', 'limited', 'english', 'useful', 'scrambled', 'weather', 'complete', 'meaning']


In [52]:
synonyms6 = []
for syn in wordnet.synsets("real", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms6.append(l.name())
syn6 = list(synonyms6)        
print(syn6)   

['real', 'existent', 'real', 'real', 'real', 'tangible', 'actual', 'genuine', 'literal', 'real', 'real', 'substantial', 'real', 'material', 'real', 'veridical', 'real']


In [53]:
for a in range(0,len(sim6)):
    for b in range(0,len(syn6)):
        try:
            tes=model.wv.similarity(w1=sim6[a],w2=syn6[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim6[a], syn6[b], tes)
        else: 
            pass

difficult real 0.9947627
difficult existent 0.9947627
difficult real 0.9947627
difficult real 0.9947627
difficult real 0.9947627
difficult tangible 0.9947627
difficult actual 0.9427825
difficult genuine 0.98221064
difficult literal 0.98221064
difficult real 0.9947627
difficult real 0.9947627
difficult substantial 0.9947627
difficult real 0.9947627
difficult material 0.9947627
difficult real 0.9947627
difficult veridical 0.9947627
difficult real 0.9947627
chaotic real 0.99403745
chaotic existent 0.99403745
chaotic real 0.99403745
chaotic real 0.99403745
chaotic real 0.99403745
chaotic tangible 0.99403745
chaotic actual 0.921247
chaotic genuine 0.9610758
chaotic literal 0.9610758
chaotic real 0.99403745
chaotic real 0.99403745
chaotic substantial 0.99403745
chaotic real 0.99403745
chaotic material 0.99403745
chaotic real 0.99403745
chaotic veridical 0.99403745
chaotic real 0.99403745
personal real 0.9935539
personal existent 0.9935539
personal real 0.9935539
personal real 0.9935539
perso

In [30]:
trait8=model.wv.most_similar("original", topn=40)
simtrait8=[]
for i in range(0,len(trait8)):   
    if wordnet.synsets(trait8[i][0], pos = wordnet.ADJ) != []:
        simtrait8.append(trait8[i][0])
    else:
        continue
sim8=list(simtrait8)
print(sim8) 

['prior', 'confirmed', 'upset', 'gone', 'wrong', 'single', 'advance', 'longer', 'young', 'senior', 'whatsoever']


In [32]:
synonyms8 = []
for syn in wordnet.synsets("original", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms8.append(l.name())
syn8 = list(synonyms8)        
print(syn8)  

['original', 'original', 'original', 'original']


In [33]:
for a in range(0,len(sim8)):
    for b in range(0,len(syn8)):
        try:
            tes=model.wv.similarity(w1=sim8[a],w2=syn8[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim8[a], syn8[b], tes)
        else: 
            pass

prior original 0.99431515
prior original 0.99431515
prior original 0.99431515
prior original 0.99431515
confirmed original 0.99299383
confirmed original 0.99299383
confirmed original 0.99299383
confirmed original 0.99299383
upset original 0.99056506
upset original 0.99056506
upset original 0.99056506
upset original 0.99056506
gone original 0.9901801
gone original 0.9901801
gone original 0.9901801
gone original 0.9901801
wrong original 0.9901359
wrong original 0.9901359
wrong original 0.9901359
wrong original 0.9901359
single original 0.98994577
single original 0.98994577
single original 0.98994577
single original 0.98994577
advance original 0.9896788
advance original 0.9896788
advance original 0.9896788
advance original 0.9896788
longer original 0.9894594
longer original 0.9894594
longer original 0.9894594
longer original 0.9894594
young original 0.9888071
young original 0.9888071
young original 0.9888071
young original 0.9888071
senior original 0.98815984
senior original 0.98815984
se

In [42]:
trait9=model.wv.most_similar("cheerful", topn=35)
simtrait9=[]
for i in range(0,len(trait9)):   
    if wordnet.synsets(trait9[i][0], pos = wordnet.ADJ) != []:
        simtrait9.append(trait9[i][0])
    else:
        continue
sim9=list(simtrait9)
print(sim9) 

['kind', 'willing', 'smiling', 'slow', 'unfriendly', 'neither', 'welcome', 'knowledgeable', 'exceptional', 'prompt']


In [43]:
synonyms9 = []
for syn in wordnet.synsets("cheerful", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms9.append(l.name())
syn9 = list(synonyms9)        
print(syn9)  

['cheerful', 'cheerful', 'pollyannaish', 'upbeat']


In [44]:
for a in range(0,len(sim9)):
    for b in range(0,len(syn9)):
        try:
            tes=model.wv.similarity(w1=sim9[a],w2=syn9[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim9[a], syn9[b], tes)
        else: 
            pass

kind cheerful 0.9929966
kind cheerful 0.9929966
kind pollyannaish 0.9929966
kind upbeat 0.9929966
willing cheerful 0.9896908
willing cheerful 0.9896908
willing pollyannaish 0.9896908
willing upbeat 0.9896908
smiling cheerful 0.9867468
smiling cheerful 0.9867468
smiling pollyannaish 0.9867468
smiling upbeat 0.9867468
slow cheerful 0.98258376
slow cheerful 0.98258376
slow pollyannaish 0.98258376
slow upbeat 0.98258376
unfriendly cheerful 0.981762
unfriendly cheerful 0.981762
unfriendly pollyannaish 0.981762
unfriendly upbeat 0.981762
neither cheerful 0.979415
neither cheerful 0.979415
neither pollyannaish 0.979415
neither upbeat 0.979415
welcome cheerful 0.97919196
welcome cheerful 0.97919196
welcome pollyannaish 0.97919196
welcome upbeat 0.97919196
knowledgeable cheerful 0.9785885
knowledgeable cheerful 0.9785885
knowledgeable pollyannaish 0.9785885
knowledgeable upbeat 0.9785885
exceptional cheerful 0.9769391
exceptional cheerful 0.9769391
exceptional pollyannaish 0.9769391
exceptional

In [46]:
trait10=model.wv.most_similar("friendly", topn=14)
simtrait10=[]
for i in range(0,len(trait10)):   
    if wordnet.synsets(trait10[i][0], pos = wordnet.ADJ) != []:
        simtrait10.append(trait10[i][0])
    else:
        continue
sim10=list(simtrait10)
print(sim10) 

['helpful', 'efficient', 'polite', 'welcoming', 'professional', 'attentive', 'pleasant', 'lovely', 'accommodating', 'approachable']


In [47]:
synonyms10 = []
for syn in wordnet.synsets("friendly", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms10.append(l.name())
syn10 = list(synonyms10)        
print(syn10) 

['friendly', 'friendly', 'favorable', 'well-disposed', 'friendly', 'friendly']


In [48]:
for a in range(0,len(sim10)):
    for b in range(0,len(syn10)):
        try:
            tes=model.wv.similarity(w1=sim10[a],w2=syn10[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim10[a], syn10[b], tes)
        else: 
            pass

helpful friendly 0.990812
helpful friendly 0.990812
helpful favorable 0.990812
helpful well-disposed 0.990812
helpful friendly 0.990812
helpful friendly 0.990812
efficient friendly 0.92557305
efficient friendly 0.92557305
efficient favorable 0.92557305
efficient well-disposed 0.92557305
efficient friendly 0.92557305
efficient friendly 0.92557305
polite friendly 0.923261
polite friendly 0.923261
polite favorable 0.923261
polite well-disposed 0.923261
polite friendly 0.923261
polite friendly 0.923261
welcoming friendly 0.9100282
welcoming friendly 0.9100282
welcoming favorable 0.9100282
welcoming well-disposed 0.9100282
welcoming friendly 0.9100282
welcoming friendly 0.9100282
professional friendly 0.8685045
professional friendly 0.8685045
professional favorable 0.8685045
professional well-disposed 0.8685045
professional friendly 0.8685045
professional friendly 0.8685045
attentive friendly 0.8646616
attentive friendly 0.8646616
attentive favorable 0.8646616
attentive well-disposed 0.8646

# Dimensi Excitement

In [65]:
trait22=model.wv.most_similar("trendy", topn=29)
simtrait22=[]
for i in range(0,len(trait22)):   
    if wordnet.synsets(trait22[i][0], pos = wordnet.ADJ) != []:
        simtrait22.append(trait22[i][0])
    else:
        continue
sim22=list(simtrait22)
print(sim22) 

['impressive', 'pretty', 'chic', 'terrific', 'interior', 'gorgeous', 'smart', 'relaxing', 'public', 'designed']


In [66]:
synonyms22 = []
for syn in wordnet.synsets("trendy", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms22.append(l.name())
syn22 = list(synonyms22)        
print(syn22)  

['trendy', 'voguish']


In [67]:
for a in range(0,len(sim22)):
    for b in range(0,len(syn22)):
        try:
            tes=model.wv.similarity(w1=sim22[a],w2=syn22[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim22[a], syn22[b], tes)
        else: 
            pass

impressive trendy 0.9797152
impressive voguish 0.9797152
pretty trendy 0.9772897
pretty voguish 0.9772897
chic trendy 0.9765283
chic voguish 0.9765283
terrific trendy 0.9744693
terrific voguish 0.9744693
interior trendy 0.9743075
interior voguish 0.9743075
gorgeous trendy 0.97402704
gorgeous voguish 0.97402704
smart trendy 0.9738841
smart voguish 0.9738841
relaxing trendy 0.97279656
relaxing voguish 0.97279656
public trendy 0.97214687
public voguish 0.97214687
designed trendy 0.972035
designed voguish 0.972035


In [51]:
trait25=model.wv.most_similar("cool", topn=34)
simtrait25=[]
for i in range(0,len(trait25)):   
    if wordnet.synsets(trait25[i][0], pos = wordnet.ADJ) != []:
        simtrait25.append(trait25[i][0])
    else:
        continue
sim25=list(simtrait25)
print(sim25) 

['biggest', 'low', 'inside', 'upper', 'larger', 'sound', 'plain', 'kindly', 'level', 'purple']


In [52]:
synonyms25 = []
for syn in wordnet.synsets("cool", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms25.append(l.name())
syn25 = list(synonyms25)        
print(syn25)

['cool', 'cool', 'coolheaded', 'nerveless', 'cool', 'cool', 'cool', 'cool']


In [53]:
for a in range(0,len(sim25)):
    for b in range(0,len(syn25)):
        try:
            tes=model.wv.similarity(w1=sim25[a],w2=syn25[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim25[a], syn25[b], tes)
        else: 
            pass

biggest cool 0.99567765
biggest cool 0.99567765
biggest coolheaded 0.99567765
biggest nerveless 0.99567765
biggest cool 0.99567765
biggest cool 0.99567765
biggest cool 0.99567765
biggest cool 0.99567765
low cool 0.9933169
low cool 0.9933169
low coolheaded 0.9933169
low nerveless 0.9933169
low cool 0.9933169
low cool 0.9933169
low cool 0.9933169
low cool 0.9933169
inside cool 0.9931154
inside cool 0.9931154
inside coolheaded 0.9931154
inside nerveless 0.9931154
inside cool 0.9931154
inside cool 0.9931154
inside cool 0.9931154
inside cool 0.9931154
upper cool 0.9928879
upper cool 0.9928879
upper coolheaded 0.9928879
upper nerveless 0.9928879
upper cool 0.9928879
upper cool 0.9928879
upper cool 0.9928879
upper cool 0.9928879
larger cool 0.99175006
larger cool 0.99175006
larger coolheaded 0.99175006
larger nerveless 0.99175006
larger cool 0.99175006
larger cool 0.99175006
larger cool 0.99175006
larger cool 0.99175006
sound cool 0.9903126
sound cool 0.9903126
sound coolheaded 0.9903126
soun

In [27]:
trait26=model.wv.most_similar("young", topn=40)
simtrait26=[]
for i in range(0,len(trait26)):   
    if wordnet.synsets(trait26[i][0], pos = wordnet.ADJ) != []:
        simtrait26.append(trait26[i][0])
    else:
        continue
sim26=list(simtrait26)
print(sim26) 

['wrong', 'regular', 'whatsoever', 'single', 'secure', 'ignored', 'lost', 'tipped', 'covered', 'understood', 'tired']


In [28]:
synonyms26 = []
for syn in wordnet.synsets("young", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms26.append(l.name())
syn26 = list(synonyms26)        
print(syn26)  

['young', 'immature', 'new', 'young', 'youthful', 'vernal', 'young', 'young', 'unseasoned', 'untested', 'untried', 'young']


In [29]:
for a in range(0,len(sim26)):
    for b in range(0,len(syn26)):
        try:
            tes=model.wv.similarity(w1=sim26[a],w2=syn26[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim26[a], syn26[b], tes)
        else: 
            pass

wrong young 0.99553454
wrong immature 0.99553454
wrong young 0.99553454
wrong youthful 0.99553454
wrong vernal 0.99553454
wrong young 0.99553454
wrong young 0.99553454
wrong unseasoned 0.99553454
wrong untested 0.99553454
wrong untried 0.99553454
wrong young 0.99553454
regular young 0.9954564
regular immature 0.9954564
regular young 0.9954564
regular youthful 0.9954564
regular vernal 0.9954564
regular young 0.9954564
regular young 0.9954564
regular unseasoned 0.9954564
regular untested 0.9954564
regular untried 0.9954564
regular young 0.9954564
whatsoever young 0.9954114
whatsoever immature 0.9954114
whatsoever young 0.9954114
whatsoever youthful 0.9954114
whatsoever vernal 0.9954114
whatsoever young 0.9954114
whatsoever young 0.9954114
whatsoever unseasoned 0.9954114
whatsoever untested 0.9954114
whatsoever untried 0.9954114
whatsoever young 0.9954114
single young 0.99534607
single immature 0.99534607
single young 0.99534607
single youthful 0.99534607
single vernal 0.99534607
single y

In [26]:
trait28=model.wv.most_similar("unique", topn=20)
simtrait28=[]
for i in range(0,len(trait28)):   
    if wordnet.synsets(trait28[i][0], pos = wordnet.ADJ) != []:
        simtrait28.append(trait28[i][0])
    else:
        continue
sim28=list(simtrait28)
print(sim28) 

['ample', 'very', 'warm', 'designed', 'equipped', 'cramped', 'standard', 'mediocre', 'smallish', 'complementary']


In [54]:
synonyms28 = []
for syn in wordnet.synsets("unique", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms28.append(l.name())
syn28 = list(synonyms28)        
print(syn28)

['alone', 'unique', 'unequaled', 'unequalled', 'unparalleled', 'unique', 'singular', 'unique', 'unique']


In [55]:
for a in range(0,len(sim28)):
    for b in range(0,len(syn28)):
        try:
            tes=model.wv.similarity(w1=sim28[a],w2=syn28[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim28[a], syn28[b], tes)
        else: 
            pass

ample alone 0.91013587
ample unique 0.97617894
ample unequaled 0.97617894
ample unequalled 0.97617894
ample unparalleled 0.97617894
ample unique 0.97617894
ample singular 0.97617894
ample unique 0.97617894
ample unique 0.97617894
very alone 0.94076943
very unique 0.9681436
very unequaled 0.9681436
very unequalled 0.9681436
very unparalleled 0.9681436
very unique 0.9681436
very singular 0.9681436
very unique 0.9681436
very unique 0.9681436
warm alone 0.89420784
warm unique 0.96618676
warm unequaled 0.96618676
warm unequalled 0.96618676
warm unparalleled 0.96618676
warm unique 0.96618676
warm singular 0.96618676
warm unique 0.96618676
warm unique 0.96618676
designed alone 0.9487597
designed unique 0.9645449
designed unequaled 0.9645449
designed unequalled 0.9645449
designed unparalleled 0.9645449
designed unique 0.9645449
designed singular 0.9645449
designed unique 0.9645449
designed unique 0.9645449
equipped alone 0.8544562
equipped unique 0.9640197
equipped unequaled 0.9640197
equipped

In [62]:
trait211=model.wv.most_similar("contemporary", topn=17)
simtrait211=[]
for i in range(0,len(trait211)):   
    if wordnet.synsets(trait211[i][0], pos = wordnet.ADJ) != []:
        simtrait211.append(trait211[i][0])
    else:
        continue
sim211=list(simtrait211)
print(sim211) 

['immaculate', 'sleek', 'decorated', 'functional', 'chic', 'funky', 'roomy', 'airy', 'massive', 'smallish']


In [63]:
synonyms211 = []
for syn in wordnet.synsets("contemporary", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms211.append(l.name())
syn211 = list(synonyms211)        
print(syn211)  

['contemporary', 'modern-day', 'contemporary', 'present-day', 'contemporaneous', 'contemporary']


In [64]:
for a in range(0,len(sim211)):
    for b in range(0,len(syn211)):
        try:
            tes=model.wv.similarity(w1=sim211[a],w2=syn211[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim211[a], syn211[b], tes)
        else: 
            pass

immaculate contemporary 0.99047995
immaculate modern-day 0.99047995
immaculate contemporary 0.99047995
immaculate present-day 0.99047995
immaculate contemporaneous 0.99047995
immaculate contemporary 0.99047995
sleek contemporary 0.9870808
sleek modern-day 0.9870808
sleek contemporary 0.9870808
sleek present-day 0.9870808
sleek contemporaneous 0.9870808
sleek contemporary 0.9870808
decorated contemporary 0.9860965
decorated modern-day 0.9860965
decorated contemporary 0.9860965
decorated present-day 0.9860965
decorated contemporaneous 0.9860965
decorated contemporary 0.9860965
functional contemporary 0.9848772
functional modern-day 0.9848772
functional contemporary 0.9848772
functional present-day 0.9848772
functional contemporaneous 0.9848772
functional contemporary 0.9848772
chic contemporary 0.98396456
chic modern-day 0.98396456
chic contemporary 0.98396456
chic present-day 0.98396456
chic contemporaneous 0.98396456
chic contemporary 0.98396456
funky contemporary 0.98368776
funky mode

# Dimensi Competence

In [69]:
trait33=model.wv.most_similar("secure", topn=40)
simtrait33=[]
for i in range(0,len(trait33)):   
    if wordnet.synsets(trait33[i][0], pos = wordnet.ADJ) != []:
        simtrait33.append(trait33[i][0])
    else:
        continue
sim33=list(simtrait33)
print(sim33) 

['regular', 'young', 'plastic', 'ignored', 'lost', 'required', 'running', 'moving', 'closed', 'unable']


In [70]:
synonyms33 = []
for syn in wordnet.synsets("secure", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms33.append(l.name())
syn33 = list(synonyms33)        
print(syn33) 

['secure', 'unafraid', 'untroubled', 'secure', 'secure', 'impregnable', 'inviolable', 'secure', 'strong', 'unassailable', 'unattackable', 'dependable', 'good', 'safe', 'secure']


In [71]:
for a in range(0,len(sim33)):
    for b in range(0,len(syn33)):
        try:
            tes=model.wv.similarity(w1=sim33[a],w2=syn33[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim33[a], syn33[b], tes)
        else: 
            pass

regular secure 0.9953923
regular unafraid 0.9953923
regular untroubled 0.9953923
regular secure 0.9953923
regular secure 0.9953923
regular impregnable 0.9953923
regular inviolable 0.9953923
regular secure 0.9953923
regular strong 0.9833766
regular unassailable 0.9833766
regular unattackable 0.9833766
regular dependable 0.9833766
regular good 0.5017771
regular safe 0.814026
regular secure 0.9953923
young secure 0.99513614
young unafraid 0.99513614
young untroubled 0.99513614
young secure 0.99513614
young secure 0.99513614
young impregnable 0.99513614
young inviolable 0.99513614
young secure 0.99513614
young strong 0.9779175
young unassailable 0.9779175
young unattackable 0.9779175
young dependable 0.9779175
young safe 0.8033867
young secure 0.99513614
plastic secure 0.994357
plastic unafraid 0.994357
plastic untroubled 0.994357
plastic secure 0.994357
plastic secure 0.994357
plastic impregnable 0.994357
plastic inviolable 0.994357
plastic secure 0.994357
plastic strong 0.9870177
plastic

In [78]:
trait36=model.wv.most_similar("corporate", topn=45)
simtrait36=[]
for i in range(0,len(trait36)):   
    if wordnet.synsets(trait36[i][0], pos = wordnet.ADJ) != []:
        simtrait36.append(trait36[i][0])
    else:
        continue
sim36=list(simtrait36)
print(sim36) 

['gone', 'wrong', 'longer', 'continued', 'bothered', 'telling', 'sent', 'yellow', 'cut', 'assured']


In [79]:
synonyms36 = []
for syn in wordnet.synsets("corporate", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms36.append(l.name())
syn36 = list(synonyms36)        
print(syn36) 

['corporate', 'bodied', 'corporal', 'corporate', 'embodied', 'incarnate', 'corporate', 'collective', 'corporate', 'incorporated']


In [80]:
for a in range(0,len(sim36)):
    for b in range(0,len(syn36)):
        try:
            tes=model.wv.similarity(w1=sim36[a],w2=syn36[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim36[a], syn36[b], tes)
        else: 
            pass

gone corporate 0.9943311
gone bodied 0.9943311
gone corporal 0.9943311
gone corporate 0.9943311
gone embodied 0.9943311
gone incarnate 0.9943311
gone corporate 0.9943311
gone collective 0.9943311
gone corporate 0.9943311
gone incorporated 0.9943311
wrong corporate 0.9939257
wrong bodied 0.9939257
wrong corporal 0.9939257
wrong corporate 0.9939257
wrong embodied 0.9939257
wrong incarnate 0.9939257
wrong corporate 0.9939257
wrong collective 0.9939257
wrong corporate 0.9939257
wrong incorporated 0.9939257
longer corporate 0.9927381
longer bodied 0.9927381
longer corporal 0.9927381
longer corporate 0.9927381
longer embodied 0.9927381
longer incarnate 0.9927381
longer corporate 0.9927381
longer collective 0.9927381
longer corporate 0.9927381
longer incorporated 0.9927381
continued corporate 0.9927244
continued bodied 0.9927244
continued corporal 0.9927244
continued corporate 0.9927244
continued embodied 0.9927244
continued incarnate 0.9927244
continued corporate 0.9927244
continued collecti

# Dimensi Sophistication

In [90]:
trait44=model.wv.most_similar("charming", topn=53)
simtrait44=[]
for i in range(0,len(trait44)):   
    if wordnet.synsets(trait44[i][0], pos = wordnet.ADJ) != []:
        simtrait44.append(trait44[i][0])
    else:
        continue
sim44=list(simtrait44)
print(sim44) 

['cleaner', 'secure', 'required', 'complete', 'moved', 'counter', 'plain', 'behind', 'assuming', 'dry']


In [91]:
synonyms44 = []
for syn in wordnet.synsets("charming", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms44.append(l.name())
syn44 = list(synonyms44)        
print(syn44) 

['charming', 'charming', 'magic', 'magical', 'sorcerous', 'witching', 'wizard', 'wizardly']


In [92]:
for a in range(0,len(sim44)):
    for b in range(0,len(syn44)):
        try:
            tes=model.wv.similarity(w1=sim44[a],w2=syn44[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim44[a], syn44[b], tes)
        else: 
            pass

cleaner charming 0.9843727
cleaner charming 0.9843727
cleaner magic 0.9843727
cleaner magical 0.9843727
cleaner sorcerous 0.9843727
cleaner witching 0.9843727
cleaner wizard 0.9843727
cleaner wizardly 0.9843727
secure charming 0.9821248
secure charming 0.9821248
secure magic 0.9821248
secure magical 0.9821248
secure sorcerous 0.9821248
secure witching 0.9821248
secure wizard 0.9821248
secure wizardly 0.9821248
required charming 0.9814305
required charming 0.9814305
required magic 0.9814305
required magical 0.9814305
required sorcerous 0.9814305
required witching 0.9814305
required wizard 0.9814305
required wizardly 0.9814305
complete charming 0.98139167
complete charming 0.98139167
complete magic 0.98139167
complete magical 0.98139167
complete sorcerous 0.98139167
complete witching 0.98139167
complete wizard 0.98139167
complete wizardly 0.98139167
moved charming 0.9813392
moved charming 0.9813392
moved magic 0.9813392
moved magical 0.9813392
moved sorcerous 0.9813392
moved witching 0.9

In [96]:
trait46=model.wv.most_similar("smooth", topn=42)
simtrait46=[]
for i in range(0,len(trait46)):   
    if wordnet.synsets(trait46[i][0], pos = wordnet.ADJ) != []:
        simtrait46.append(trait46[i][0])
    else:
        continue
sim46=list(simtrait46)
print(sim46) 

['swift', 'simple', 'behind', 'speedy', 'straightforward', 'checked', 'seamless', 'surly', 'requested', 'earlier']


In [97]:
synonyms46 = []
for syn in wordnet.synsets("smooth", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms46.append(l.name())
syn46 = list(synonyms46)        
print(syn46) 

['smooth', 'politic', 'smooth', 'suave', 'bland', 'smooth', 'fluent', 'fluid', 'liquid', 'smooth', 'legato', 'smooth', 'smooth', 'smooth', 'placid', 'quiet', 'still', 'tranquil', 'smooth', 'unruffled']


In [98]:
for a in range(0,len(sim46)):
    for b in range(0,len(syn46)):
        try:
            tes=model.wv.similarity(w1=sim46[a],w2=syn46[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim46[a], syn46[b], tes)
        else: 
            pass

swift smooth 0.971607
swift politic 0.971607
swift smooth 0.971607
swift suave 0.971607
swift bland 0.9295468
swift smooth 0.971607
swift fluent 0.971607
swift fluid 0.971607
swift liquid 0.971607
swift smooth 0.971607
swift legato 0.971607
swift smooth 0.971607
swift smooth 0.971607
swift smooth 0.971607
swift placid 0.971607
swift quiet 0.7728401
swift still 0.8844898
swift tranquil 0.8844898
swift smooth 0.971607
swift unruffled 0.971607
simple smooth 0.96207094
simple politic 0.96207094
simple smooth 0.96207094
simple suave 0.96207094
simple bland 0.9049592
simple smooth 0.96207094
simple fluent 0.96207094
simple fluid 0.96207094
simple liquid 0.96207094
simple smooth 0.96207094
simple legato 0.96207094
simple smooth 0.96207094
simple smooth 0.96207094
simple smooth 0.96207094
simple placid 0.96207094
simple quiet 0.8207731
simple still 0.89953446
simple tranquil 0.89953446
simple smooth 0.96207094
simple unruffled 0.96207094
behind smooth 0.9529767
behind politic 0.9529767
behind 

# Dimensi Ruggedness

In [100]:
trait51=model.wv.most_similar("outdoor", topn=40)
simtrait51=[]
for i in range(0,len(trait51)):   
    if wordnet.synsets(trait51[i][0], pos = wordnet.ADJ) != []:
        simtrait51.append(trait51[i][0])
    else:
        continue
sim51=list(simtrait51)
print(sim51) 

['american', 'preferred', 'organized', 'difficult', 'real', 'cheap', 'unusual', 'soft', 'handy', 'usual']


In [101]:
synonyms51 = []
for syn in wordnet.synsets("outdoor", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms51.append(l.name())
syn51 = list(synonyms51)        
print(syn51) 

['outdoor', 'out-of-door', 'outside', 'outdoor']


In [102]:
for a in range(0,len(sim51)):
    for b in range(0,len(syn51)):
        try:
            tes=model.wv.similarity(w1=sim51[a],w2=syn51[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim51[a], syn51[b], tes)
        else: 
            pass

american outdoor 0.991425
american out-of-door 0.991425
american outside 0.9542729
american outdoor 0.991425
preferred outdoor 0.98935235
preferred out-of-door 0.98935235
preferred outside 0.9522581
preferred outdoor 0.98935235
organized outdoor 0.98846734
organized out-of-door 0.98846734
organized outside 0.91806245
organized outdoor 0.98846734
difficult outdoor 0.9878263
difficult out-of-door 0.9878263
difficult outside 0.96123534
difficult outdoor 0.9878263
real outdoor 0.9852891
real out-of-door 0.9852891
real outside 0.945912
real outdoor 0.9852891
cheap outdoor 0.9852253
cheap out-of-door 0.9852253
cheap outside 0.9369099
cheap outdoor 0.9852253
unusual outdoor 0.98507947
unusual out-of-door 0.98507947
unusual outside 0.9476475
unusual outdoor 0.98507947
soft outdoor 0.9847852
soft out-of-door 0.9847852
soft outside 0.9381326
soft outdoor 0.9847852
handy outdoor 0.98418045
handy out-of-door 0.98418045
handy outside 0.9749187
handy outdoor 0.98418045
usual outdoor 0.98399985
usual

# Dimensi Sustainability

In [110]:
trait61=model.wv.most_similar("lovely", topn=17)
simtrait61=[]
for i in range(0,len(trait61)):   
    if wordnet.synsets(trait61[i][0], pos = wordnet.ADJ) != []:
        simtrait61.append(trait61[i][0])
    else:
        continue
sim61=list(simtrait61)
print(sim61) 

['nice', 'clean', 'super', 'tidy', 'spotless', 'modern', 'beautiful', 'cozy', 'bright', 'well']


In [111]:
synonyms61 = []
for syn in wordnet.synsets("lovely", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms61.append(l.name())
syn61 = list(synonyms61)        
print(syn61) 

['lovely', 'adorable', 'endearing', 'lovely']


In [112]:
for a in range(0,len(sim61)):
    for b in range(0,len(syn61)):
        try:
            tes=model.wv.similarity(w1=sim61[a],w2=syn61[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim61[a], syn61[b], tes)
        else: 
            pass

nice lovely 0.97896165
nice adorable 0.97896165
nice endearing 0.97896165
nice lovely 0.97896165
clean lovely 0.9498893
clean adorable 0.9498893
clean endearing 0.9498893
clean lovely 0.9498893
super lovely 0.9396349
super adorable 0.9396349
super endearing 0.9396349
super lovely 0.9396349
tidy lovely 0.9367731
tidy adorable 0.9367731
tidy endearing 0.9367731
tidy lovely 0.9367731
spotless lovely 0.9354369
spotless adorable 0.9354369
spotless endearing 0.9354369
spotless lovely 0.9354369
modern lovely 0.9294517
modern adorable 0.9294517
modern endearing 0.9294517
modern lovely 0.9294517
beautiful lovely 0.926663
beautiful adorable 0.926663
beautiful endearing 0.926663
beautiful lovely 0.926663
cozy lovely 0.91048366
cozy adorable 0.91048366
cozy endearing 0.91048366
cozy lovely 0.91048366
bright lovely 0.9029461
bright adorable 0.9029461
bright endearing 0.9029461
bright lovely 0.9029461
well lovely 0.88980496
well adorable 0.88980496
well endearing 0.88980496
well lovely 0.88980496


In [128]:
trait62=model.wv.most_similar("giving", topn=26)
simtrait62=[]
for i in range(0,len(trait62)):   
    if wordnet.synsets(trait62[i][0], pos = wordnet.ADJ) != []:
        simtrait62.append(trait62[i][0])
    else:
        continue
sim62=list(simtrait62)
print(sim62) 

['stopped', 'lucky', 'honest', 'ended', 'must', 'treated', 'unprofessional', 'packed', 'filled', 'arranged']


In [123]:
synonyms62 = []
for syn in wordnet.synsets("giving", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms62.append(l.name())
syn62 = list(synonyms62)        
print(syn62) 

['big', 'bighearted', 'bounteous', 'bountiful', 'freehanded', 'handsome', 'giving', 'liberal', 'openhanded']


In [129]:
for a in range(0,len(sim62)):
    for b in range(0,len(syn62)):
        try:
            tes=model.wv.similarity(w1=sim62[a],w2=syn62[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim62[a], syn62[b], tes)
        else: 
            pass

stopped big 0.65010226
stopped bighearted 0.65010226
stopped bounteous 0.65010226
stopped bountiful 0.65010226
stopped freehanded 0.65010226
stopped handsome 0.65010226
stopped giving 0.9972161
stopped liberal 0.9972161
stopped openhanded 0.9972161
lucky big 0.6678822
lucky bighearted 0.6678822
lucky bounteous 0.6678822
lucky bountiful 0.6678822
lucky freehanded 0.6678822
lucky handsome 0.6678822
lucky giving 0.9964427
lucky liberal 0.9964427
lucky openhanded 0.9964427
honest big 0.6860405
honest bighearted 0.6860405
honest bounteous 0.6860405
honest bountiful 0.6860405
honest freehanded 0.6860405
honest handsome 0.6860405
honest giving 0.995635
honest liberal 0.995635
honest openhanded 0.995635
ended big 0.66691434
ended bighearted 0.66691434
ended bounteous 0.66691434
ended bountiful 0.66691434
ended freehanded 0.66691434
ended handsome 0.66691434
ended giving 0.995191
ended liberal 0.995191
ended openhanded 0.995191
must big 0.6568612
must bighearted 0.6568612
must bounteous 0.65686

In [135]:
trait65=model.wv.most_similar("fair", topn=25)
simtrait65=[]
for i in range(0,len(trait65)):   
    if wordnet.synsets(trait65[i][0], pos = wordnet.ADJ) != []:
        simtrait65.append(trait65[i][0])
    else:
        continue
sim65=list(simtrait65)
print(sim65) 

['otherwise', 'whole', 'similar', 'more', 'picky', 'sufficient', 'cheap', 'personal', 'important', 'entire']


In [136]:
synonyms65 = []
for syn in wordnet.synsets("fair", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms65.append(l.name())
syn65 = list(synonyms65)        
print(syn65) 

['fair', 'just', 'fair', 'fairish', 'reasonable', 'bonny', 'bonnie', 'comely', 'fair', 'sightly', 'fair', 'average', 'fair', 'mediocre', 'middling', 'fair', 'clean', 'fair', 'honest', 'fair', 'fair', 'fair', 'fairish']


In [137]:
for a in range(0,len(sim65)):
    for b in range(0,len(syn65)):
        try:
            tes=model.wv.similarity(w1=sim65[a],w2=syn65[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim65[a], syn65[b], tes)
        else: 
            pass

otherwise fair 0.9953872
otherwise just 0.94866574
otherwise fair 0.9953872
otherwise fairish 0.9953872
otherwise reasonable 0.9193891
otherwise bonny 0.9193891
otherwise bonnie 0.9193891
otherwise comely 0.9193891
otherwise fair 0.9953872
otherwise sightly 0.9953872
otherwise fair 0.9953872
otherwise average 0.94662976
otherwise fair 0.9953872
otherwise mediocre 0.92519474
otherwise middling 0.92519474
otherwise fair 0.9953872
otherwise clean 0.50132835
otherwise fair 0.9953872
otherwise honest 0.9880051
otherwise fair 0.9953872
otherwise fair 0.9953872
otherwise fair 0.9953872
otherwise fairish 0.9953872
whole fair 0.9920691
whole just 0.93642074
whole fair 0.9920691
whole fairish 0.9920691
whole reasonable 0.9056313
whole bonny 0.9056313
whole bonnie 0.9056313
whole comely 0.9056313
whole fair 0.9920691
whole sightly 0.9920691
whole fair 0.9920691
whole average 0.9565935
whole fair 0.9920691
whole mediocre 0.9327043
whole middling 0.9327043
whole fair 0.9920691
whole clean 0.5867942

In [139]:
trait67=model.wv.most_similar("aware", topn=41)
simtrait67=[]
for i in range(0,len(trait67)):   
    if wordnet.synsets(trait67[i][0], pos = wordnet.ADJ) != []:
        simtrait67.append(trait67[i][0])
    else:
        continue
sim67=list(simtrait67)
print(sim67) 

['blanket', 'stated', 'noted', 'whatsoever', 'plastic', 'awkward', 'loud', 'taking', 'electrical', 'required']


In [140]:
synonyms67 = []
for syn in wordnet.synsets("aware", pos = wordnet.ADJ):
    for l in syn.lemmas():
        synonyms67.append(l.name())
syn67 = list(synonyms67)        
print(syn67) 

['aware', 'cognizant', 'cognisant', 'mindful', 'aware']


In [141]:
for a in range(0,len(sim67)):
    for b in range(0,len(syn67)):
        try:
            tes=model.wv.similarity(w1=sim67[a],w2=syn67[b])
        except KeyError: 
            pass
        if tes > 0.5:
            print(sim67[a], syn67[b], tes)
        else: 
            pass

blanket aware 0.9974653
blanket cognizant 0.9974653
blanket cognisant 0.9974653
blanket mindful 0.9974653
blanket aware 0.9974653
stated aware 0.9974475
stated cognizant 0.9974475
stated cognisant 0.9974475
stated mindful 0.9974475
stated aware 0.9974475
noted aware 0.9952106
noted cognizant 0.9952106
noted cognisant 0.9952106
noted mindful 0.9952106
noted aware 0.9952106
whatsoever aware 0.99487853
whatsoever cognizant 0.99487853
whatsoever cognisant 0.99487853
whatsoever mindful 0.99487853
whatsoever aware 0.99487853
plastic aware 0.99481136
plastic cognizant 0.99481136
plastic cognisant 0.99481136
plastic mindful 0.99481136
plastic aware 0.99481136
awkward aware 0.9947903
awkward cognizant 0.9947903
awkward cognisant 0.9947903
awkward mindful 0.9947903
awkward aware 0.9947903
loud aware 0.99451876
loud cognizant 0.99451876
loud cognisant 0.99451876
loud mindful 0.99451876
loud aware 0.99451876
taking aware 0.99449414
taking cognizant 0.99449414
taking cognisant 0.99449414
taking min

In [143]:
df['content'] = df['content'].astype("str").astype("string")

# Similarity with Hotel, Staff, Room

In [30]:
model.wv.most_similar("hotel", topn=300)

[('place', 0.9586787223815918),
 ('enjoyed', 0.9474710822105408),
 ('loved', 0.9175904989242554),
 ('best', 0.910925567150116),
 ('brilliant', 0.9079177975654602),
 ('pros', 0.9065639972686768),
 ('perfect', 0.9059151411056519),
 ('ny', 0.9054049849510193),
 ('location', 0.9004500508308411),
 ('wonderful', 0.8995833396911621),
 ('overall', 0.898104190826416),
 ('visiting', 0.8944746255874634),
 ('fantastic', 0.8938037753105164),
 ('brand', 0.8894022703170776),
 ('enjoyable', 0.8890149593353271),
 ('nyc', 0.8743764758110046),
 ('fabulous', 0.8742343187332153),
 ('excellent', 0.8643581867218018),
 ('convenient', 0.8632642030715942),
 ('chose', 0.8625158667564392),
 ('value', 0.8621460795402527),
 ('thank', 0.8617674112319946),
 ('great', 0.8614494800567627),
 ('love', 0.8576070666313171),
 ('everything', 0.8571925759315491),
 ('recommended', 0.852756679058075),
 ('here', 0.8526121377944946),
 ('thoroughly', 0.8513391017913818),
 ('condition', 0.8505871891975403),
 ('gem', 0.8486745357513

In [ ]:
# hotel, riu, place, plaza,property, garden, theater, decor, bar, interior    

In [31]:
model.wv.most_similar("staff", topn=300)

[('helpful', 0.9908664226531982),
 ('friendly', 0.9836352467536926),
 ('polite', 0.9414870142936707),
 ('efficient', 0.9294572472572327),
 ('professional', 0.901592493057251),
 ('welcoming', 0.8974874019622803),
 ('pleasant', 0.8868393301963806),
 ('courteous', 0.8829128742218018),
 ('attentive', 0.8779811859130859),
 ('really', 0.86702561378479),
 ('always', 0.8666353225708008),
 ('helpfull', 0.8568854928016663),
 ('service', 0.8557998538017273),
 ('extremely', 0.8425599336624146),
 ('approachable', 0.8386121988296509),
 ('accommodating', 0.8337449431419373),
 ('especially', 0.8291931748390198),
 ('lovely', 0.8209136128425598),
 ('lobby', 0.7847312688827515),
 ('super', 0.7844955325126648),
 ('knowledgeable', 0.7786766886711121),
 ('happy', 0.7772942781448364),
 ('throughout', 0.7757574319839478),
 ('cheerful', 0.7747924327850342),
 ('reception', 0.7697721123695374),
 ('excellent', 0.7637258768081665),
 ('knowledgable', 0.7601665258407593),
 ('willing', 0.7595812082290649),
 ('help', 

In [ ]:
# staff, service, reception, bellboys, receptionists, waiters, boys, doormen, concierge, porters, catering 

In [32]:
model.wv.most_similar("room", topn=300)

[('double', 0.954907238483429),
 ('king', 0.9490435719490051),
 ('deluxe', 0.9353919625282288),
 ('queen', 0.930720329284668),
 ('sofa', 0.9142757058143616),
 ('high', 0.9085163474082947),
 ('beds', 0.8948485851287842),
 ('bed', 0.8929225206375122),
 ('suite', 0.8865376710891724),
 ('ceiling', 0.8859963417053223),
 ('doubles', 0.875032901763916),
 ('twin', 0.8726879954338074),
 ('small', 0.8715015053749084),
 ('two', 0.8685393333435059),
 ('sized', 0.8607843518257141),
 ('bugs', 0.8514161705970764),
 ('higher', 0.8492627739906311),
 ('space', 0.8431701064109802),
 ('reserved', 0.8414021134376526),
 ('pushed', 0.8398635983467102),
 ('delighted', 0.8397648930549622),
 ('cleaned', 0.8382115364074707),
 ('balcony', 0.8377710580825806),
 ('sleeper', 0.8372114896774292),
 ('floor', 0.8364836573600769),
 ('side', 0.8324470520019531),
 ('rd', 0.8323615789413452),
 ('comfy', 0.8310055136680603),
 ('adults', 0.8299787044525146),
 ('adequate', 0.8289984464645386),
 ('upgrade', 0.8246141672134399)

In [ ]:
# room, double, bed, space, balcony, pillows, mattress, rooms, toiletries, bedroom, decoration, bathroom